# MPS Hamiltonian Learning with TensorKrowch

In [4]:
import numpy as np
import torch
from torchvision import transforms, datasets
import tensorkrowch as tk

import pandas as pd
import glob
import yaml

### Things to do:

This will be a pytorch version of the Hamiltonian learning problem, where the NN has a layer with an MPS structure from tensorkrowch. There are a few things that I need to sort out:
1. This NN doesn't train from data, it starts from an ansatz and modifies it until it finds the optimal solution
2. I still need to run the dynamics for every epoch, which consist on:
    2.1. Taking initial state and applyting rotations in the X,Y and Z directions, with the option to customize which rotations I want to apply
    2.2. Doing a time evolution of the resulting state under a Hamiltonian. The Hamiltonian contains only interaction terms, and it also must be customizable
3. After running the Hamiltonian, we extract bitstring probabilities and compute nll loss function with the input data, which are the generated bitstrings 

## Data loading

In [5]:
def load_config(config_path):
    '''Load configuration from YAML file'''
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

def load_experimental_data(config):
    """Load experimental/simulated data"""
    N = config["L"]
    chi = config['bond_dimension']
    T_max = config["t_max"]
    search_pattern = f"../data/experimental_data_quantum_sampling_L{N}_Chi_{chi}_*_counts.csv"
    files = glob.glob(search_pattern)

    if not files:
        raise FileNotFoundError(f"No data found for L={N}")

    config_file = files[0]
    file_core = config_file.replace(".csv", "").replace("../data/experimental_data_quantum_sampling_", "")
    
    print(f"\n{'='*60}")
    print(f"LOADING DATA: {file_core}")
    print(f"{'='*60}")
    
    df_counts = pd.read_csv(f"../data/experimental_data_quantum_sampling_{file_core}.csv")
        
    # Remove leading single quote if present
    if df_counts['bitstring'].astype(str).str.startswith("'").all():
        df_counts['bitstring'] = df_counts['bitstring'].str[1:]
    
    # Now extract values
    bitstrings = df_counts['bitstring'].values.astype(str)
    counts_shots = df_counts['count'].values.astype(np.int32)
    
    return bitstrings, counts_shots

## Useful functions

In [ ]:
def paulis(dtype=torch.complex64):
    '''Creates single-qubit basis operators'''
    sx = torch.tensor([[0., 1.], [1., 0.]], dtype=dtype)
    sy = torch.tensor([[0., -1j], [1j, 0.]], dtype=dtype)
    sz = torch.tensor([[1., 0.], [0., -1.]], dtype=dtype)
    id2 = torch.eye(2, dtype=dtype)
    return sx, sy, sz, id2

def kron_n(ops):
    '''Tensor product of a list of operators'''
    out = ops[0]
    for A in ops[1:]:
        out = torch.kron(out, A)
    return out


def x_rotation(theta, dtype=torch.complex64):
    sx = torch.tensor([[0., 1.], [1., 0.]], dtype=dtype)
    return torch.matrix_exp(-1j * theta / 2 * sx)

def y_rotation(theta, dtype=torch.complex64):
    sy = torch.tensor([[0., -1j], [1j, 0.]], dtype=dtype)
    return torch.matrix_exp(-1j * theta / 2 * sy)

def z_rotation(theta, dtype=torch.complex64):
    sz = torch.tensor([[1., 0.], [0., -1.]], dtype=dtype)
    return torch.matrix_exp(-1j * theta / 2 * sz)

In [15]:
def prepare_initial_state(L, kind, dtype=torch.complex64):
    """Prepare initial quantum states for L qubits."""
    if kind == 'all_zeros':
        psi0 = torch.zeros(2**L, dtype=dtype)
        psi0[0] = 1.0
        
    elif kind == 'all_plus':
        plus = torch.ones(2, dtype=dtype) / np.sqrt(2)
        psi0 = plus
        for _ in range(L - 1):
            psi0 = torch.kron(psi0, plus)
            
    else:
        raise ValueError(f"Initial state '{kind}' not recognized. "
                        f"Use 'all_zeros' or 'all_plus'")
    return psi0

In [19]:
class OperatorClass:
    '''Class that contains a list of all the operator types the Hamiltonian will have
       The operators will be applied to each qubit, and we will allow for the construction of any
       combination of Pauli strings 
    '''
    def __init__(self, L, dtype=torch.complex64):

        self.L = L
        self.dim = 2**L
        self.pauli_basis = {}
        self.pauli_basis['X'], self.pauli_basis['Y'], self.pauli_basis['Z'], self.pauli_basis['I'] = paulis(dtype)
        self.operators = []
    
    def __len__(self):
        return len(self.operators)
    
    def __getitem__(self, idx):
        return self.operators[idx]
    
    def add_operators(self, pauli_string:str):
        #e.g. 'X','Y','ZZ'
        '''Adds one type of operator at a time. It loops through all the qubits, 
        and for each position does the tensor product of the whole chain, with the 
        required qubits substituted by the operators of the string'''

        if len(pauli_string) > self.L:
            raise ValueError(f"Pauli string '{pauli_string}' longer than system size {self.L}")
        
        if not all(char in 'XYZI' for char in pauli_string):
            raise ValueError(f"Invalid character in '{pauli_string}'. Use only X, Y, Z, I")
        
        for i in range(self.L - len(pauli_string) + 1):
                #Create identity operators for each qubit
                ops = [self.pauli_basis['I']]*self.L
                for j, char in enumerate(pauli_string):
                     #Build string
                     ops[i+j] = self.pauli_basis[char]
                self.operators.append(kron_n(ops))
        print(f"{pauli_string} terms added to the Hamiltonian")

## Manual NN

These were the functions defined by Marcin

In [22]:
def mlp_forward(params, x):
    h = x
    for layer in params[:-1]:
        h = np.tanh(h @ layer["W"] + layer["b"])
    last = params[-1]
    return h @ last["W"] + last["b"]


def init_mlp_params(layer_sizes, scale=0.1):
    params = []
    for i, (m, n) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        # Initialize weights with scaled normal distribution
        W = scale * torch.randn((m, n))
        # Initialize biases to zero
        b = torch.zeros((n,))
        params.append({"W": W, "b": b})
    return params

## Pytorch NN

Equivalent of Marcin's functions but with Pytorch syntax

## TensorKrowch NN

My pytorch NN integrating MPS layer from TensorKrowch

In [23]:
config_file = "/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/tensorkrowch_version/config/MPS_learning_configuration.yaml"

#load configuration
print(config_file)
CONFIG = load_config(config_file)

# Load data
bitstrings, counts_shots = load_experimental_data(CONFIG)

L = CONFIG['L']
inital_state_kind = CONFIG['initial_state_kind']
dim = 2**L

psi0 = prepare_initial_state(L, inital_state_kind)

#Initialize and onfigure Hamiltonian Ansatz
OPS_LIST = OperatorClass(L)

OPS_LIST.add_operators('X')
OPS_LIST.add_operators('Z')
OPS_LIST.add_operators('ZZ')

NUM_COEFFICIENTS = len(OPS_LIST)
NUM_ROTATIONS = L * (CONFIG['x_fields'] + CONFIG['y_fields'] + CONFIG['z_fields'])

#Initialize parameters
torch.manual_seed(CONFIG["seed_init"])

theta_init = torch.rand(NUM_COEFFICIENTS, dtype=torch.float32)
rot_init = torch.rand(NUM_ROTATIONS, dtype=torch.float32)

# Initialize NN
NN_MAP_FUN = mlp_forward
NN_INPUT_DIM = 1  # Time-dependent
NN_OUTPUT_DIM = NUM_COEFFICIENTS  # NN only outputs Hamiltonian corrections

layer_sizes = [NN_INPUT_DIM] + CONFIG["NN_hidden_sizes"] + [NN_OUTPUT_DIM]

nn_params = init_mlp_params(layer_sizes, scale=0.1)

params = {"theta": theta_init, "nn": nn_params}


/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/tensorkrowch_version/config/MPS_learning_configuration.yaml

LOADING DATA: L4_Chi_5_R50000_counts
X terms added to the Hamiltonian
Z terms added to the Hamiltonian
ZZ terms added to the Hamiltonian
